In [1]:
import logging
import time
import ray
import random
from random import randint
import numpy as np

In [2]:
from collections import defaultdict
@ray.remote
class LoggingActor(object):
    def __init__(self):
        # create a container of dictionaries.
        self.logs = defaultdict(list)
    
    # log the message for a particular experiment in its 
    # respective dictionary
    def log(self, index, message):
        self.logs[index].append(message)
    
    # fetch all logs as collection
    def get_logs(self):
        return dict(self.logs)
    
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(9):
        # pretend this is an experiment that produces a nine results for 
        # experiment result; in our case it's just a simple message 
        # Push a logging message to the actor.
        exp_key = f"experiment-{experiment_index}"
        logging_actor.log.remote(exp_key, 'On iteration {}'.format(i))  

In [3]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.1
Dashboard:,http://127.0.0.1:8265


Run three experiments.

In [4]:
logging_actor = LoggingActor.remote()
experiment_ids = []
# Run three different experiments
for i in range(3):
    experiment_ids.append(run_experiment.remote(i, logging_actor))

### Fetch the results 

For each experement, we will have 9 iteration results

In [5]:
for i in range(3):
    time.sleep(1)
    logs = logging_actor.get_logs.remote()
    print(ray.get(logs))

{'experiment-0': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8'], 'experiment-1': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8'], 'experiment-2': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8']}
{'experiment-0': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8'], 'experiment-1': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8'], 'experiment-2': ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On itera

In [6]:
ray.shutdown()